In [1]:
##############################
#### Forest plot
##############################
library(data.table) # version 1.14.6
library(meta) # version 6.2-1
library(grid) # version 4.2.2
library(scales) # version 1.2.1

Loading 'meta' package (version 4.18-2).
Type 'help(meta)' for a brief overview.



In [2]:
# set working directory
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/epi/")

In [4]:
#### Athero
## Forest Plot
glm_athero <- fread("final_glm.multivariable_atherogenic_lipid.incident_ch.2023Jul07.csv", header = T)
table(glm_athero$Exposure)
glm_athero <- subset(glm_athero, glm_athero$Exposure %in% c("Dyslipidemia", "tg_to_hdl_base_INT", 
                                                            "sex_by_smoking"))

glm_athero$Exposure[glm_athero$Exposure=="Dyslipidemia"] <- "(b) Dyslipidemia"

glm_athero$Exposure[glm_athero$Exposure=="tg_to_hdl_base_INT"] <- "(a) TG/HDL-C ratio"

glm_athero$Exposure[glm_athero$Exposure=="sex_by_smoking"] <- "(c) Male sex: Ever Smoker"

table(glm_athero$Exposure)



          Dyslipidemia           hdl_base_low ldl_base_nomal_vs_high 
                     6                      6                      6 
        sex_by_smoking     tg_to_hdl_base_INT 
                     6                      6 


       (a) TG/HDL-C ratio          (b) Dyslipidemia (c) Male sex: Ever Smoker 
                        6                         6                         6 

In [5]:

glm_athero$Outcome[glm_athero$Outcome=="incident_CH"] <- "Overall CH"
glm_athero$Outcome[glm_athero$Outcome=="incident_DNMT3A"] <- "DNMT3A"
glm_athero$Outcome[glm_athero$Outcome=="incident_TET2"] <- "TET2"
glm_athero$Outcome[glm_athero$Outcome=="incident_ASXL1"] <- "ASXL1"
glm_athero$Outcome[glm_athero$Outcome=="incident_SF"] <- "SF"
glm_athero$Outcome[glm_athero$Outcome=="incident_DDR"] <- "DDR"

table(glm_athero$Outcome)

glm_athero$Outcome <- ordered(glm_athero$Outcome, 
                                   levels = c("Overall CH", 
                                              "DNMT3A", "TET2", 
                                              "ASXL1", 
                                              "SF", "DDR")) 

## exposure
glm_athero$Exposure <- ordered(glm_athero$Exposure, 
                                    levels = c("(a) TG/HDL-C ratio", 
                                               "(b) Dyslipidemia", 
                                               "(c) Male sex: Ever Smoker"))
## Format
glm_athero$P_val <- formatC(x = glm_athero$P, digits = 1,format = "E")

# OR
glm_athero$OR <- formatC(round(exp(glm_athero$Beta),2), digits = 2, format = "f")

glm_athero$lSE <- ( glm_athero$Beta - 1.96 * glm_athero$SE)
glm_athero$uSE <- ( glm_athero$Beta + 1.96 * glm_athero$SE)

# 95% CI
glm_athero$CI95 <- paste0("[",formatC(round(exp( glm_athero$Beta - 1.96 * glm_athero$SE),2), digits = 2, format = "f"),
                               ", ",
                               formatC(round(exp( glm_athero$Beta + 1.96 * glm_athero$SE),2), digits = 2, format = "f"), 
                               "]")
head(glm_athero)

glm_athero$sig <-  ifelse(glm_athero$P<0.0025, "***","")



     ASXL1        DDR     DNMT3A Overall CH         SF       TET2 
         3          3          3          3          3          3 

Dataset,Outcome,Exposure,Beta,SE,t-stat,P,P_val,OR,lSE,uSE,CI95
<chr>,<ord>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
Adjusted,Overall CH,(a) TG/HDL-C ratio,-0.06189474,0.07962889,-0.7772900,0.43698767,4.4E-01,0.94,-0.21796737,0.09417788,"[0.80, 1.10]"
Adjusted,Overall CH,(b) Dyslipidemia,0.02739987,0.23700563,0.1156085,0.90796279,9.1E-01,1.03,-0.43713115,0.49193090,"[0.65, 1.64]"
Adjusted,DNMT3A,(a) TG/HDL-C ratio,-0.09781986,0.11350737,-0.8617930,0.38880142,3.9E-01,0.91,-0.32029432,0.12465459,"[0.73, 1.13]"
Adjusted,DNMT3A,(b) Dyslipidemia,-0.59542872,0.40048983,-1.4867512,0.13708055,1.4E-01,0.55,-1.38038879,0.18953135,"[0.25, 1.21]"
Adjusted,TET2,(a) TG/HDL-C ratio,0.12326462,0.15450663,0.7977950,0.42498945,4.2E-01,1.13,-0.17956837,0.42609761,"[0.84, 1.53]"
Adjusted,TET2,(b) Dyslipidemia,0.75082835,0.36590362,2.0519839,0.04017123,4.0E-02,2.12,0.03365725,1.46799945,"[1.03, 4.34]"


In [6]:

##### Adjusted model: Athero lipid
# Gender by smoking status
b_athero <- metagen(TE = Beta,
                     lower = lSE,
                     upper = uSE,
                     studlab = Outcome,
                     subgroup=Exposure,
                     data=glm_athero,
                     sm="OR")


pdf("figS6.final_glm.athero.Forest_incidentCH.2023Jul07.pdf",
    width = 8, height= 8)
forest(x = b_athero, 
       common=F, 
       random=F, 
       hetstat=F, 
       subgroup=k.w>=1, 
       weight.study="same",  
       level=0.95, 
       xlim=c(0.5, 3), 
       smlab="Effect of Exposures\non Incident CH\n", 
       smlab.pos=0, 
       colgap=unit(7, "mm"),
       xlab="Odds Ratio", 
       squaresize=0.6, 
       col.subgroup="black", 
       colgap.left=unit(0.1,"cm"),
       colgap.forest.left="3mm", 
       colgap.forest.right="2mm", 
       leftcols=c("studlab"), 
       leftlabs = c("                     "),
       rightcols=c("OR","CI95","P_val", "sig"),
       rightlabs=c("OR","95% CI","P", ""),
       #rightcols=NULL, 
       #rightlabs=NULL,
       col.inside="black", 
       plotwidth=unit(6.5, "cm"), 
       print.subgroup.name=F)
dev.off()


ERROR: Error in metagen(TE = Beta, lower = lSE, upper = uSE, studlab = Outcome, : unused argument (subgroup = Exposure)


In [3]:
# W/O adjusting for chol lowering med
#### Athero
## Forest Plot
glm_athero <- fread("final_glm.multivariable_atherogenic_lipid.incident_ch.nocholMed.2023Jul12.csv", header = T)
table(glm_athero$Exposure)
glm_athero <- subset(glm_athero, glm_athero$Exposure %in% c("Dyslipidemia", "tg_to_hdl_base_INT", 
                                                            "sex_by_smoking"))

glm_athero$Exposure[glm_athero$Exposure=="Dyslipidemia"] <- "(b) Dyslipidemia"

glm_athero$Exposure[glm_athero$Exposure=="tg_to_hdl_base_INT"] <- "(a) TG/HDL-C ratio"

glm_athero$Exposure[glm_athero$Exposure=="sex_by_smoking"] <- "(c) Male sex: Ever Smoker"

table(glm_athero$Exposure)


          Dyslipidemia           hdl_base_low ldl_base_nomal_vs_high 
                     6                      6                      6 
    tg_to_hdl_base_INT 
                     6 


(a) TG/HDL-C ratio   (b) Dyslipidemia 
                 6                  6 

In [4]:

glm_athero$Outcome[glm_athero$Outcome=="incident_CH"] <- "Overall CH"
glm_athero$Outcome[glm_athero$Outcome=="incident_DNMT3A"] <- "DNMT3A"
glm_athero$Outcome[glm_athero$Outcome=="incident_TET2"] <- "TET2"
glm_athero$Outcome[glm_athero$Outcome=="incident_ASXL1"] <- "ASXL1"
glm_athero$Outcome[glm_athero$Outcome=="incident_SF"] <- "SF"
glm_athero$Outcome[glm_athero$Outcome=="incident_DDR"] <- "DDR"

table(glm_athero$Outcome)

glm_athero$Outcome <- ordered(glm_athero$Outcome, 
                                   levels = c("Overall CH", 
                                              "DNMT3A", "TET2", 
                                              "ASXL1", 
                                              "SF", "DDR")) 

## exposure
glm_athero$Exposure <- ordered(glm_athero$Exposure, 
                                    levels = c("(a) TG/HDL-C ratio", 
                                               "(b) Dyslipidemia", 
                                               "(c) Male sex: Ever Smoker"))
## Format
glm_athero$P_val <- formatC(x = glm_athero$P, digits = 1,format = "E")

# OR
glm_athero$OR <- formatC(round(exp(glm_athero$Beta),2), digits = 2, format = "f")

glm_athero$lSE <- ( glm_athero$Beta - 1.96 * glm_athero$SE)
glm_athero$uSE <- ( glm_athero$Beta + 1.96 * glm_athero$SE)

# 95% CI
glm_athero$CI95 <- paste0("[",formatC(round(exp( glm_athero$Beta - 1.96 * glm_athero$SE),2), digits = 2, format = "f"),
                               ", ",
                               formatC(round(exp( glm_athero$Beta + 1.96 * glm_athero$SE),2), digits = 2, format = "f"), 
                               "]")
head(glm_athero)

glm_athero$sig <-  ifelse(glm_athero$P<0.0025, "***","")



     ASXL1        DDR     DNMT3A Overall CH         SF       TET2 
         2          2          2          2          2          2 

Dataset,Outcome,Exposure,Beta,SE,t-stat,P,P_val,OR,lSE,uSE,CI95
<chr>,<ord>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
Adjusted_no_chol_med_base,Overall CH,(a) TG/HDL-C ratio,-0.05686754,0.07952767,-0.7150661,0.474568211,4.7E-01,0.94,-0.2127418,0.09900669,"[0.81, 1.10]"
Adjusted_no_chol_med_base,Overall CH,(b) Dyslipidemia,0.17756399,0.20663566,0.8593095,0.390169782,3.9E-01,1.19,-0.2274419,0.58256988,"[0.80, 1.79]"
Adjusted_no_chol_med_base,DNMT3A,(a) TG/HDL-C ratio,-0.09644310,0.11328991,-0.8512947,0.394605662,3.9E-01,0.91,-0.3184913,0.12560512,"[0.73, 1.13]"
Adjusted_no_chol_med_base,DNMT3A,(b) Dyslipidemia,-0.40867530,0.36002420,-1.1351329,0.256319657,2.6E-01,0.66,-1.1143227,0.29697213,"[0.33, 1.35]"
Adjusted_no_chol_med_base,TET2,(a) TG/HDL-C ratio,0.13280747,0.15401996,0.8622744,0.388536542,3.9E-01,1.14,-0.1690717,0.43468660,"[0.84, 1.54]"
Adjusted_no_chol_med_base,TET2,(b) Dyslipidemia,0.83455919,0.31443224,2.6541782,0.007950184,8.0E-03,2.30,0.2182720,1.45084638,"[1.24, 4.27]"


In [ ]:

##### Adjusted model: Athero lipid
# Gender by smoking status
b_athero <- metagen(TE = Beta,
                     lower = lSE,
                     upper = uSE,
                     studlab = Outcome,
                     subgroup=Exposure,
                     data=glm_athero,
                     sm="OR")


pdf("figS6.final_glm.athero.Forest_incidentCH.nocholMed.2023Jul12.pdf",
    width = 8, height= 8)
forest(x = b_athero, 
       common=F, 
       random=F, 
       hetstat=F, 
       subgroup=k.w>=1, 
       weight.study="same",  
       level=0.95, 
       xlim=c(0.5, 3), 
       smlab="Effect of Exposures\non Incident CH\n", 
       smlab.pos=0, 
       colgap=unit(7, "mm"),
       xlab="Odds Ratio", 
       squaresize=0.6, 
       col.subgroup="black", 
       colgap.left=unit(0.1,"cm"),
       colgap.forest.left="3mm", 
       colgap.forest.right="2mm", 
       leftcols=c("studlab"), 
       leftlabs = c("                     "),
       rightcols=c("OR","CI95","P_val", "sig"),
       rightlabs=c("OR","95% CI","P", ""),
       #rightcols=NULL, 
       #rightlabs=NULL,
       col.inside="black", 
       plotwidth=unit(6.5, "cm"), 
       print.subgroup.name=F)
dev.off()


### Final Analysis: 
Sex by Smoking Status adjusted for all covariates

TG/HDL-C and Dyslipidemia analysis were adjusted for:  

In [7]:
glm_sex_by_smoking <- fread("final_glm.multivariable_atherogenic_lipid.incident_ch.2023Jul07.csv", header = T)
table(glm_sex_by_smoking$Exposure)
glm_sex_by_smoking <- subset(glm_sex_by_smoking, glm_sex_by_smoking$Exposure == "sex_by_smoking")
table(glm_sex_by_smoking$Exposure)


          Dyslipidemia           hdl_base_low ldl_base_nomal_vs_high 
                     6                      6                      6 
        sex_by_smoking     tg_to_hdl_base_INT 
                     6                      6 


sex_by_smoking 
             6 

In [8]:
# w/o chol_med_base adjustment, nonHDL_base_INT, hdl_base_INT
#### Athero
## Forest Plot
glm_athero <- fread("final_glm.multivariable_atherogenic_lipid.incident_ch.noChol_hdl_nonHdl.2023Jul12.csv", header = T)
table(glm_athero$Exposure)



          Dyslipidemia           hdl_base_low ldl_base_nomal_vs_high 
                     6                      6                      6 
    tg_to_hdl_base_INT 
                     6 

In [9]:
glm_athero <- as.data.frame(rbind(glm_athero,glm_sex_by_smoking),stringsAsFactors = F)
table(glm_athero$Exposure)
str(glm_athero)


          Dyslipidemia           hdl_base_low ldl_base_nomal_vs_high 
                     6                      6                      6 
        sex_by_smoking     tg_to_hdl_base_INT 
                     6                      6 

'data.frame':	30 obs. of  7 variables:
 $ Dataset : chr  "Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT" "Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT" "Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT" "Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT" ...
 $ Outcome : chr  "incident_CH" "incident_CH" "incident_CH" "incident_CH" ...
 $ Exposure: chr  "tg_to_hdl_base_INT" "ldl_base_nomal_vs_high" "Dyslipidemia" "hdl_base_low" ...
 $ Beta    : num  -0.027 0.1907 0.2037 -0.0688 -0.0362 ...
 $ SE      : num  0.0479 0.107 0.2034 0.0921 0.0686 ...
 $ t-stat  : num  -0.563 1.783 1.001 -0.747 -0.527 ...
 $ P       : num  0.5732 0.0746 0.3166 0.4553 0.598 ...


In [10]:
# w/o chol_med_base adjustment, nonHDL_base_INT, hdl_base_INT
#### Athero
## Forest Plot
# glm_athero <- fread("final_glm.multivariable_atherogenic_lipid.incident_ch.noChol_hdl_nonHdl.2023Jul12.csv", header = T)
# table(glm_athero$Exposure)
glm_athero <- subset(glm_athero, glm_athero$Exposure %in% c("Dyslipidemia", "tg_to_hdl_base_INT", 
                                                            "sex_by_smoking"))

glm_athero$Exposure[glm_athero$Exposure=="Dyslipidemia"] <- "(b) Dyslipidemia"

glm_athero$Exposure[glm_athero$Exposure=="tg_to_hdl_base_INT"] <- "(a) TG/HDL-C ratio"

glm_athero$Exposure[glm_athero$Exposure=="sex_by_smoking"] <- "(c) Male sex: Ever Smoker"

table(glm_athero$Exposure)




       (a) TG/HDL-C ratio          (b) Dyslipidemia (c) Male sex: Ever Smoker 
                        6                         6                         6 

In [11]:

glm_athero$Outcome[glm_athero$Outcome=="incident_CH"] <- "Overall CH"
glm_athero$Outcome[glm_athero$Outcome=="incident_DNMT3A"] <- "DNMT3A"
glm_athero$Outcome[glm_athero$Outcome=="incident_TET2"] <- "TET2"
glm_athero$Outcome[glm_athero$Outcome=="incident_ASXL1"] <- "ASXL1"
glm_athero$Outcome[glm_athero$Outcome=="incident_SF"] <- "SF"
glm_athero$Outcome[glm_athero$Outcome=="incident_DDR"] <- "DDR"

table(glm_athero$Outcome)

glm_athero$Outcome <- ordered(glm_athero$Outcome, 
                                   levels = c("Overall CH", 
                                              "DNMT3A", "TET2", 
                                              "ASXL1", 
                                              "SF", "DDR")) 

## exposure
glm_athero$Exposure <- ordered(glm_athero$Exposure, 
                                    levels = c("(a) TG/HDL-C ratio", 
                                               "(b) Dyslipidemia", 
                                               "(c) Male sex: Ever Smoker"))
## Format
glm_athero$P_val <- formatC(x = glm_athero$P, digits = 1,format = "E")

# OR
glm_athero$OR <- formatC(round(exp(glm_athero$Beta),2), digits = 2, format = "f")

glm_athero$lSE <- ( glm_athero$Beta - 1.96 * glm_athero$SE)
glm_athero$uSE <- ( glm_athero$Beta + 1.96 * glm_athero$SE)

# 95% CI
glm_athero$CI95 <- paste0("[",formatC(round(exp( glm_athero$Beta - 1.96 * glm_athero$SE),2), digits = 2, format = "f"),
                               ", ",
                               formatC(round(exp( glm_athero$Beta + 1.96 * glm_athero$SE),2), digits = 2, format = "f"), 
                               "]")
head(glm_athero)

glm_athero$sig <-  ifelse(glm_athero$P<0.0025, "***","")



     ASXL1        DDR     DNMT3A Overall CH         SF       TET2 
         3          3          3          3          3          3 

,Dataset,Outcome,Exposure,Beta,SE,t-stat,P,P_val,OR,lSE,uSE,CI95
,<chr>,<ord>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT,Overall CH,(a) TG/HDL-C ratio,-0.02697057,0.04787701,-0.5633304,0.573209942,5.7E-01,0.97,-0.120809512,0.06686837,"[0.89, 1.07]"
3,Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT,Overall CH,(b) Dyslipidemia,0.20365188,0.20335014,1.0014839,0.316592925,3.2E-01,1.23,-0.194914381,0.60221815,"[0.82, 1.83]"
5,Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT,DNMT3A,(a) TG/HDL-C ratio,-0.03616976,0.06859358,-0.5273053,0.597981562,6.0E-01,0.96,-0.170613179,0.09827366,"[0.84, 1.10]"
7,Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT,DNMT3A,(b) Dyslipidemia,-0.35864202,0.35695052,-1.0047388,0.315022659,3.2E-01,0.70,-1.058265037,0.34098099,"[0.35, 1.41]"
9,Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT,TET2,(a) TG/HDL-C ratio,0.17342850,0.09211844,1.8826686,0.059745298,6.0E-02,1.19,-0.007123651,0.35398065,"[0.99, 1.42]"
11,Adjusted_no_chol_med_base_nonHDL_base_INT_hdl_base_INT,TET2,(b) Dyslipidemia,0.91896686,0.30340822,3.0288133,0.002455164,2.5E-03,2.51,0.324286743,1.51364698,"[1.38, 4.54]"


In [ ]:

##### Adjusted model: Athero lipid
# Gender by smoking status
b_athero <- metagen(TE = Beta,
                     lower = lSE,
                     upper = uSE,
                     studlab = Outcome,
                     subgroup=Exposure,
                     data=glm_athero,
                     sm="OR")


pdf("figS6.final_glm.athero.Forest_incidentCH.noChol_hdl_nonHdl.2023Jul12.pdf",
    width = 8, height= 7)
forest(x = b_athero, 
       common=F, 
       random=F, 
       hetstat=F, 
       subgroup=k.w>=1, 
       weight.study="same",  
       level=0.95, 
       xlim=c(0.5, 3), 
       smlab="Effect of Exposures\non Incident CH\n", 
       smlab.pos=0, 
       colgap=unit(7, "mm"),
       xlab="Odds Ratio", 
       squaresize=0.6, 
       col.subgroup="black", 
       colgap.left=unit(0.1,"cm"),
       colgap.forest.left="3mm", 
       colgap.forest.right="2mm", 
       leftcols=c("studlab"), 
       leftlabs = c("                     "),
       rightcols=c("OR","CI95","P_val", "sig"),
       rightlabs=c("OR","95% CI","P", ""),
       #rightcols=NULL, 
       #rightlabs=NULL,
       col.inside="black", 
       plotwidth=unit(6.5, "cm"), 
       print.subgroup.name=F)
dev.off()
